In [55]:
import nltk
print('hello world')

hello world


In [56]:
def foo():
    print('foo')

In [57]:
foo()

foo


In [58]:
import pandas
#pandas.read_excel(r'C:\Users\n1555085\Downloads\Copy of CSAT Help hub April Responses.xlsx')
sheet = pandas.read_excel(r'C:\Users\n1555085\Downloads\May Help Hub Data.xlsx')
courtesy = sheet['Unnamed: 9'].dropna().values.tolist()[1:]
effectiveness = sheet['Unnamed: 10'].dropna().values.tolist()[1:]
timeliness = sheet['Unnamed: 11'].dropna().values.tolist()[1:]
understanding = sheet['Unnamed: 12'].dropna().values.tolist()[1:]
nps = sheet['Unnamed: 13'].dropna().values.tolist()[1:]
comments = courtesy + effectiveness + timeliness + understanding + nps
date = sheet['Unnamed: 1'].dropna().values.tolist()[1:]
#should add completion rate

In [59]:
print(f'# of courtesy comments: {len(courtesy)}')
print(f'# of effectiveness comments: {len(effectiveness)}')
print(f'# of timeliness comments: {len(timeliness)}')
print(f'# of understanding comments: {len(understanding)}')
print(f'# of nps comments: {len(nps)}')
print(f'# of comments: {len(comments)}')

# of courtesy comments: 10
# of effectiveness comments: 17
# of timeliness comments: 24
# of understanding comments: 13
# of nps comments: 148
# of comments: 212


In [60]:
print('hi')

hi


In [62]:
#read from corpus, remove stopwords
from nltk.corpus import stopwords
from string import punctuation

with open(r'C:\Users\n1555085\Downloads\Project\positiveComments.txt', 'r') as f:
    posReviews = f.readlines()
with open(r'C:\Users\n1555085\Downloads\Project\negativeComments.txt', 'r') as f:
    negReviews = f.readlines()
print(len(posReviews), len(negReviews))

sw = set(stopwords.words('english') + list(punctuation))
notStopwords = ['not', 'no', '!', 'but', 'too', 'have', 'had']

def removeStopwords(review):
    #review.translate(None, string.punctuation)
    return ' '.join([word for word in review.split() if word not in sw or word in notStopwords])
posReviews = list(filter(lambda s: s , list(map(removeStopwords, posReviews))))
negReviews = list(filter(lambda s: s , list(map(removeStopwords, negReviews))))

posReviews[0:5]
#negReviews[0:5]

4553 1574


['Spenser Cameron set assisted IT issues. He exceptional trying assist newcomer new systems.',
 'second consultant awesome appropriate follow well letting know break fix consultants suggestion since I could not receive emails. She best.',
 'support personnel helpful patient issue I',
 'fast service.',
 'Greeting robotic, however loosened conversation went on.']

In [63]:
#define pos and neg bag-of words, and vocabulary

posWords = [word.lower() for review in posReviews for word in review.split()]
negWords = [word.lower() for review in negReviews for word in review.split()]
vocabulary = list(set(posWords + negWords))
#vocabulary[0:100]

In [64]:
#define training data: list of (review list of words, label) tuples
trainingData = [(r.split(), 'pos') for r in posReviews] + [(r.split(), 'neg') for r in negReviews]

In [72]:
#NAIVE BAYES CLASSIFIER: extract feature vector from each review, train classifier
def featureVector(reviewSplit):
    reviewWords = set(reviewSplit)
    features = {}
    for word in vocabulary:
        features[word] = word in reviewWords
    return features

naiveBayesClassifier = nltk.NaiveBayesClassifier.train(nltk.classify.apply_features(featureVector, trainingData))

#classify functions
def NBclassify(review):
    review = removeStopwords(review)
    features = featureVector(review.split())
    probDist = naiveBayesClassifier.prob_classify(features)
    confidence = max(probDist.prob('pos'), probDist.prob('neg'))
    #pos and neg add to 1
    return (naiveBayesClassifier.classify(features).upper(), confidence)

def NBclassifyComments(comments):
    return [(c, NBclassify(c)) for c in comments]

In [104]:
NBlabeledComments = NBclassifyComments(effectiveness)
NBlabeledComments

[('1st ticket they marked resolved even though it was not.',
  ('POS', 0.7108082493822837)),
 ('no next steps. have to restart my request from scratch.',
  ('POS', 0.6788194278729774)),
 ('See previous comment.', ('POS', 0.9998479614398925)),
 ('No response nor assistance was given', ('POS', 0.9996127782321947)),
 ('I was given a solution that did not meet my needs. Finally I was directed to a JIRA site for another team and my ticket was resolved although my issue still remains. Hopefully the JIRA team can help.',
  ('NEG', 0.943626558667604)),
 ('there is a fix without calling internet which i was told to do',
  ('POS', 0.9696059036741314)),
 ('.', ('POS', 0.9998884231699515)),
 ('I was asked multiple times to keep doing the same thing by different people. Clear my cache and cookies when I literally had just done that 1 minute prior. That was a bit frustrating.',
  ('NEG', 0.9999973632610897)),
 ('Unfortunately recreating my VDE is not an ideal scenario. Having said that, it has nothi

In [119]:
#SVM Classifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8, sublinear_tf = True, use_idf = True)
train_vectors = vectorizer.fit_transform(posReviews + negReviews)

classifier_linear = SVC(kernel='linear')
classifier_linear.fit(train_vectors, ['pos'] * len(posReviews) + ['neg'] * len(negReviews))

classifier_poly = SVC(kernel='poly')
classifier_poly.fit(train_vectors, ['pos'] * len(posReviews) + ['neg'] * len(negReviews))

classifier_rbf = SVC(kernel='rbf')
classifier_rbf.fit(train_vectors, ['pos'] * len(posReviews) + ['neg'] * len(negReviews))

classifier_sigmoid = SVC(kernel='sigmoid')
classifier_sigmoid.fit(train_vectors, ['pos'] * len(posReviews) + ['neg'] * len(negReviews))

# svm kernel can be ‘linear’, ‘poly’, ‘rbf’, or ‘sigmoid’
def SVMclassify(review, kernel):
    review = removeStopwords(review)
    review_vector = vectorizer.transform([review]) # vectorizing
    if kernel == 'linear':
        return classifier_linear.predict(review_vector)[0]
    elif kernel == 'poly':
        return classifier_poly.predict(review_vector)[0]
    elif kernel == 'rbf':
        return classifier_rbf.predict(review_vector)[0]
    elif kernel == 'sigmoid':
        return classifier_sigmoid.predict(review_vector)[0]
    return None

def SVMclassifyComments(comments, kernel):
    return [(c, SVMclassify(c, kernel)) for c in comments]

In [124]:
SVMlabelledComments = SVMclassifyComments(timeliness, 'rbf')
SVMlabelledComments

[('not resolved over 2 weeks later', 'neg'),
 ('It took a couple of days for the ticket to be addressed but this was fine as it was not blocking my work.',
  'neg'),
 ('Issue was not resolved. Never connected with anyone before they closed the ticket.',
  'neg'),
 ('The fixes that were implemented by the help technician fixed my problem!',
  'pos'),
 ('This access request takes too long for processing. This took aver a week. Access to SMART is never going to be accidental, no on e even knows what SMART is unless you actually need it. This has been a major pain point for years but that doesnt seem to affect anyone but the one who made the request in the first place.',
  'neg'),
 ('It would be great if resolution will be quicker', 'pos'),
 ('No response nor assistance was given', 'neg'),
 ('the first agent reached out at 8pm local time and never responded back to me when I replied the next day. Eventually I was contacted by someone else.',
  'neg'),
 ('The first person who helped me coul